# MLP (Multi-Layer Perceptron) 학습

DeepCF 논문(AAAI 2019)의 CFNet-ml 모델을 PyTorch로 구현한 학습 노트북입니다.

## 모델 개요

MLP는 user-item embedding을 concatenate한 후 MLP로 학습하는 metric learning 기반 모델입니다.

$$\hat{y}_{ui} = \sigma(W^T(f([r_u, r_i])) + b)$$

- $r_u \in \{0,1\}^{|I|}$: user $u$의 상호작용 벡터
- $r_i \in \{0,1\}^{|U|}$: item $i$의 상호작용 벡터
- $f$: MLP (Multi-Layer Perceptron)
- $[\cdot, \cdot]$: Concatenation

## 1. 임포트 및 환경 설정

In [1]:
import sys
sys.path.append('..')

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from time import time
import random
import os

# 공통 유틸리티 임포트
from common.data_utils import load_deepcf_data, get_train_matrix
from common.train_utils import get_train_instances, TrainDataset
from common.eval_utils import evaluate_model

# CFNet-ml (MLP) 모델 임포트
from cfnet_ml.mlp_model import MLP

# 재현성을 위한 시드 고정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# 디바이스 설정 (CUDA > MPS > CPU)
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_name = f"CUDA ({torch.cuda.get_device_name(0)})"
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    device_name = "MPS (Apple Silicon)"
else:
    device = torch.device('cpu')
    device_name = "CPU"

print(f"Device: {device_name}")

Device: MPS (Apple Silicon)


## 2. 하이퍼파라미터 설정

In [2]:
# 데이터 설정
DATA_PATH = '../datasets/'
DATASET = 'ml-1m'
USE_SAMPLE = True
SAMPLE_USERS = 100

# 모델 구조 (논문 기본값)
LAYERS = [512, 256, 128, 64]  # MLP 레이어 크기
                              # layers[0]//2 = 256이 embedding 크기

# 학습 파라미터
EPOCHS = 20
BATCH_SIZE = 256
NUM_NEG = 4  # Negative sample 개수
LEARNING_RATE = 0.001  # MLP는 0.001 사용 (DMF는 0.0001)
LEARNER = 'adam'

# 평가 및 저장
TOP_K = 10
VERBOSE = 1  # 평가 주기 (epoch)
SAVE_MODEL = True
MODEL_DIR = '../pretrain/'

print(f"Dataset: {DATASET} (Sample: {USE_SAMPLE})")
print(f"Model: MLP{LAYERS} (Embedding size: {LAYERS[0]//2})")
print(f"Training: {EPOCHS} epochs, batch={BATCH_SIZE}, neg={NUM_NEG}, lr={LEARNING_RATE}")

Dataset: ml-1m (Sample: True)
Model: MLP[512, 256, 128, 64] (Embedding size: 256)
Training: 20 epochs, batch=256, neg=4, lr=0.001


## 3. 데이터 준비

샘플 데이터가 필요한 경우 `../data_sampling.ipynb`를 먼저 실행하세요.

**데이터 포맷:**
- `train.rating`, `test.rating`: `userID\titemID\trating\ttimestamp`
- `test.negative`: `(userID,itemID)\tneg1\tneg2\t...` (99개)

In [3]:
dataset_name = f'{DATASET}-sample{SAMPLE_USERS}' if USE_SAMPLE else DATASET

print(f"Loading dataset: {dataset_name}...")
t1 = time()
train, testRatings, testNegatives, num_users, num_items = load_deepcf_data(
    DATA_PATH, dataset_name
)
print(f"Loaded in {time()-t1:.1f}s")
print(f"  Users: {num_users}, Items: {num_items}")
print(f"  Train: {train.nnz}, Test: {len(testRatings)}")

Loading dataset: ml-1m-sample100...
Loaded in 0.1s
  Users: 100, Items: 2591
  Train: 17361, Test: 93


## 4. 모델 생성

**MLP 구조:**
- User Embedding: $|I| \rightarrow 256$ (linear)
- Item Embedding: $|U| \rightarrow 256$ (linear)
- Concatenation: $[256, 256] \rightarrow 512$
- MLP: $512 \rightarrow 256 \rightarrow 128 \rightarrow 64$ (ReLU)
- Prediction: $64 \rightarrow 1$ (Sigmoid)

In [4]:
# Train matrix 변환 (sparse -> dense)
train_matrix = get_train_matrix(train)

# 모델 초기화
model = MLP(train_matrix, num_users, num_items, LAYERS).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"Model: {sum(p.numel() for p in model.parameters()):,} parameters")
print(f"Optimizer: {LEARNER}, LR: {LEARNING_RATE}")

Model: 861,953 parameters
Optimizer: adam, LR: 0.001


## 5. 초기 성능 평가

**평가 메트릭:**
- Hit Ratio@K: Top-K 내 정답 포함 비율
- NDCG@K: Normalized Discounted Cumulative Gain

In [5]:
print("Evaluating initial performance...")
t1 = time()
hits, ndcgs = evaluate_model(model, testRatings, testNegatives, TOP_K, device)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print(f"Init: HR@{TOP_K}={hr:.4f}, NDCG@{TOP_K}={ndcg:.4f} [{time()-t1:.1f}s]")

best_hr, best_ndcg, best_iter = hr, ndcg, -1

Evaluating initial performance...
Init: HR@10=0.2258, NDCG@10=0.1105 [0.3s]


## 6. 학습

**학습 과정:**
1. Negative sampling: 각 positive에 대해 $N$ 개의 negative 샘플 생성
2. Mini-batch SGD로 BCE loss 최소화
   $$\mathcal{L} = -\sum_{(u,i)} y_{ui} \log \hat{y}_{ui} + (1-y_{ui}) \log(1-\hat{y}_{ui})$$
3. 매 epoch 평가 및 best model 저장

In [6]:
# 모델 저장 디렉토리 생성
if SAVE_MODEL:
    os.makedirs(MODEL_DIR, exist_ok=True)
    # 데이터셋 이름을 prefix로 사용 (예: ml-1m-sample100-ml.pth)
    model_out_file = f'{MODEL_DIR}{dataset_name}-ml.pth'

print("\nStarting training...")
print("=" * 80)

for epoch in range(EPOCHS):
    t1 = time()
    
    # Negative sampling
    user_input, item_input, labels = get_train_instances(train, NUM_NEG, num_items)
    train_dataset = TrainDataset(user_input, item_input, labels)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # Training
    model.train()
    total_loss = 0
    for batch_users, batch_items, batch_labels in train_loader:
        batch_users = batch_users.to(device)
        batch_items = batch_items.to(device)
        batch_labels = batch_labels.to(device).unsqueeze(1)
        
        predictions = model(batch_users, batch_items)
        loss = criterion(predictions, batch_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    t2 = time()
    
    # Evaluation
    if epoch % VERBOSE == 0:
        hits, ndcgs = evaluate_model(model, testRatings, testNegatives, TOP_K, device)
        hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
        
        print(f'Epoch {epoch:2d} [{t2-t1:.1f}s]: HR={hr:.4f}, NDCG={ndcg:.4f}, '
              f'loss={avg_loss:.4f} [{time()-t2:.1f}s]')
        
        # Best model 저장
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch
            if SAVE_MODEL:
                torch.save(model.state_dict(), model_out_file)
                print(f'  → Best model saved (HR={hr:.4f})')

print("=" * 80)
print("Training completed!")


Starting training...
Epoch  0 [1.2s]: HR=0.3763, NDCG=0.2000, loss=0.3311 [0.1s]
  → Best model saved (HR=0.3763)
Epoch  1 [0.9s]: HR=0.4086, NDCG=0.1979, loss=0.1725 [0.1s]
  → Best model saved (HR=0.4086)
Epoch  2 [0.9s]: HR=0.3978, NDCG=0.2081, loss=0.1094 [0.1s]
Epoch  3 [0.9s]: HR=0.3548, NDCG=0.1894, loss=0.1040 [0.1s]
Epoch  4 [0.9s]: HR=0.3548, NDCG=0.1656, loss=0.0740 [0.1s]
Epoch  5 [0.9s]: HR=0.3978, NDCG=0.1954, loss=0.0524 [0.1s]
Epoch  6 [0.9s]: HR=0.4086, NDCG=0.2023, loss=0.0444 [0.1s]
Epoch  7 [0.9s]: HR=0.4086, NDCG=0.1873, loss=0.0474 [0.1s]
Epoch  8 [1.0s]: HR=0.4086, NDCG=0.1843, loss=0.0337 [0.1s]
Epoch  9 [0.9s]: HR=0.3333, NDCG=0.1559, loss=0.0323 [0.1s]
Epoch 10 [0.9s]: HR=0.3978, NDCG=0.1812, loss=0.0314 [0.1s]
Epoch 11 [0.9s]: HR=0.3871, NDCG=0.1890, loss=0.0281 [0.1s]
Epoch 12 [0.9s]: HR=0.3763, NDCG=0.1800, loss=0.0249 [0.1s]
Epoch 13 [0.9s]: HR=0.3763, NDCG=0.1717, loss=0.0242 [0.1s]
Epoch 14 [0.9s]: HR=0.3656, NDCG=0.2023, loss=0.0255 [0.1s]
Epoch 15 [0.

## 7. 결과

In [7]:
print("\n" + "=" * 80)
print("FINAL RESULTS")
print("=" * 80)
print(f"Best Epoch: {best_iter}")
print(f"Best HR@{TOP_K}: {best_hr:.4f}")
print(f"Best NDCG@{TOP_K}: {best_ndcg:.4f}")

if SAVE_MODEL:
    print(f"\nModel saved: {model_out_file}")

print("\nConfiguration:")
print(f"  Dataset: {dataset_name}")
print(f"  Architecture: MLP{LAYERS}")
print(f"  Embedding size: {LAYERS[0]//2}")
print(f"  Training: {EPOCHS} epochs, batch={BATCH_SIZE}, neg={NUM_NEG}")
print(f"  Optimizer: {LEARNER}, lr={LEARNING_RATE}")
print("=" * 80)


FINAL RESULTS
Best Epoch: 15
Best HR@10: 0.4194
Best NDCG@10: 0.2231

Model saved: ../pretrain/ml-1m-sample100-ml.pth

Configuration:
  Dataset: ml-1m-sample100
  Architecture: MLP[512, 256, 128, 64]
  Embedding size: 256
  Training: 20 epochs, batch=256, neg=4
  Optimizer: adam, lr=0.001
